### Génération d'un tableau de coordonnées fictives dans l'attente de vraies données


In [14]:
import numpy as np

molecules_nb = 10

# Génération d'un vecteur de positions. Pour chaque molécule, le vecteur contient 3 coordonnées x, y, z,
# chacune dans l'intervalle [-1,1[
positions = np.random.random_sample(molecules_nb*3) * 2 - 1
positions

array([-0.54160352, -0.39901984,  0.08772178, -0.44709833,  0.73373848,
        0.67177955,  0.54482004,  0.27201387, -0.48761669,  0.99920236,
       -0.43867843, -0.56472728,  0.8286265 , -0.91978798,  0.32063854,
       -0.6842396 , -0.91226324,  0.47758616, -0.82044854, -0.04096083,
        0.76354988, -0.01464192, -0.00572377,  0.13096125,  0.38678974,
        0.87696817, -0.84781277, -0.90261056,  0.4908653 ,  0.80496882])

### Génération de bruit suivant une loi normale sur le tableau de coordonnées

In [25]:
sigma_norm = 0.5

# Définition d'une fonction ajoutant un bruit gaussien à un tableau de données
def noise_gen(positions, sigma):
    return positions + np.random.normal(0, sigma)

positions_bruit = noise_gen(positions, sigma_norm)
positions_bruit

array([ 0.76277324,  1.04380314,  1.16829254,  0.40391199,  0.61583541,
        0.9577566 ,  0.38040451,  0.08941694, -0.27231043,  0.85532755,
        0.84810673, -0.27752027,  1.17210846, -0.16659665,  1.47976816,
       -0.40749625, -0.11690155, -0.08060797, -0.14649729,  0.51938873,
       -0.25622362,  0.53982904, -0.40128851,  0.73892777,  0.24672635,
        0.04534827, -0.29806181,  0.96184031,  1.08353121,  1.19783559])

### Calcul du RMSE des données initiales

In [29]:
from sklearn.metrics import mean_squared_error

mean_squared_error(positions, positions_bruit)

0.23746480124601782

### Simulation des prédictions

In [27]:
# On définit une fonction simulant les prédictions d'un modèle entraîné. Précisément, on prend le jeu
# d'étiquettes et on rajoute un bruit de largeur n fois inférieure à la largeur appliquée au jeu d'entraînement
def simul_appr(true, sigma_init, facteur):
    return noise_gen(true, sigma_init/facteur)

pred = simul_appr(positions, sigma_norm, 5)

### Calcul du RMSE des prédictions

In [28]:
mean_squared_error(positions, pred)

0.020770151776989744

## Mise en situation dans le cas de l'entraînement du RN

On va définir ici des fonctions permettant de calculer le RMSE qui servira de fonction de coût et du score R2 qui servira de métrique d'évaluation du modèle, qui seront utilisées par l'algorithme d'apprentissage du RN.

Les couches d'entrée et de sortie du RN étant fixes, on doit passer par ces fonctions pour évaluer le coût et la performance du modèle sur les données représentant la sortie attendue pour le nombre d'atomes en entrée. En d'autres termes, on ne va pas forcer le modèle à prédire des distances de 0 entre des atomes non existants pour la molécule courante et les atomes fictifs du repère. De même, on ne va pas évaluer les prédictions du RN sur les distances concernant des atomes inexistants.

#### Affichage Tensor

In [ ]:
init_op = tf.initialize_all_variables()
    
    
    with tf.Session() as sess:
        sess.run(init_op) #execute init_op
        print (sess.run(targets))
        print (sess.run(predictions))
        print (sess.run(targets_masked))
        print (sess.run(predictions_masked))
        print (sess.run(RMSE))
        print(sess.run(sq_diff))

#### Calcul du masque des atomes définis en fonction du vecteur targets

In [121]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


#### Calcul du rmse

In [119]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    

    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    
    # On masque les prédictions et les étiquettes selon le masque des atomes définis
    targets_masked = tf.boolean_mask(targets, defined_mask)
    predictions_masked = tf.boolean_mask(predictions, defined_mask)
    
    
    return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Calcul du score R2 (score d'évaluation du modèle)

In [129]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    # Calcul de l'erreur totale
    total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets))))
                
    # Calcul de l'erreur inexpliquée
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets, predictions)))

    init_op = tf.initialize_all_variables()
    
    r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
 
    